In [1]:
import cv2
import numpy as np
import tensorflow as tf
tf.compat.v1.disable_eager_execution() # Speed
from tensorflow.keras.models import Model, load_model, model_from_json
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, \
                                    Conv2DTranspose, BatchNormalization, \
                                    UpSampling2D, Concatenate

In [2]:
from tensorflow.keras.utils import Sequence
class DataGen(Sequence):
    def __init__(self, x, y, batch_size):
        self.x, self.y = x, y
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        x_batch = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        y_batch = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

        # 1:1, 4:3, 16:9, 2:1
        # shapes = [(512, 384), (384, 512), (512, 288), (288, 512), (512, 256), (256, 512), (256, 256), (256, 256)]
        # shape = shapes[np.random.choice(len(shapes))]
        shape = (256, 256)
        
        x_ret, y_ret = [], []
        for x_file_name, y_file_name in zip(x_batch, y_batch):
            x_img = cv2.resize(cv2.imread(x_file_name, 0), shape)
            y_img = cv2.resize(cv2.imread(y_file_name, cv2.IMREAD_UNCHANGED), shape)
            probs = y_img[:,:,3] / 255.0
            alpha_map = np.zeros((*probs.shape, 2))
            alpha_map[:,:,0] = 1 - probs
            alpha_map[:,:,1] = probs
            x_ret.append(x_img[:,:,np.newaxis])
            y_ret.append(alpha_map)

        return np.array(x_ret), np.array(y_ret)

In [3]:
def LikeUnet():
    # input
    im = Input(shape=(None, None, 1))
    # conv1
    x = Conv2D(16, 3, padding='same', activation='relu')(im)
    conv1 = MaxPooling2D(2)(x)
    # conv2
    x = Conv2D(32, 3, padding='same', activation='relu')(conv1)
    conv2 = MaxPooling2D(2)(x)
    # conv3
    x = Conv2D(64, 3, padding='same', activation='relu')(conv2)
    conv3 = MaxPooling2D(2)(x)
    # conv4
    x = Conv2D(128, 3, padding='same', activation='relu')(conv3)
    conv4 = MaxPooling2D(2)(x)
    # conv5
    x = Conv2D(256, 3, padding='same', activation='relu')(conv4)
    x = MaxPooling2D(2)(x)
    # BatchNormalization
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    # trans-conv1
    x = Conv2D(256, 3, padding='same', activation='relu')(x)
    x = Conv2DTranspose(128, 3, strides=2, padding='same', activation='relu')(x)
    # trans-conv2
    x = Concatenate()([conv4, x])
    x = Conv2D(128, 3, padding='same', activation='relu')(x)
    x = Conv2DTranspose(64, 3, strides=2, padding='same', activation='relu')(x)
    # trans-conv3
    x = Concatenate()([conv3, x])
    x = Conv2D(64, 3, padding='same', activation='relu')(x)
    x = Conv2DTranspose(32, 3, strides=2, padding='same', activation='relu')(x)
    # trans-conv4
    x = Concatenate()([conv2, x])
    x = Conv2D(32, 3, padding='same', activation='relu')(x)
    x = Conv2DTranspose(16, 3, strides=2, padding='same', activation='relu')(x)
    # trans-conv5
    x = Concatenate()([conv1, x])
    x = Conv2D(16, 3, padding='same', activation='relu')(x)
    x = Conv2DTranspose(8, 3, strides=2, padding='same', activation='relu')(x)
    # output
    x = Concatenate([im, x])
    x = Conv2D(16, 3, padding='same', activation='relu')(x)
    out = Conv2D(2, 1, padding='same', activation='softmax')(x)
    return Model(inputs=im, outputs=out)

In [4]:
with open("x_train.txt", "r") as f:
    x_train = [filename[:-1] for filename in f.readlines()]
with open("y_train.txt", "r") as f:
    y_train = [filename[:-1] for filename in f.readlines()]
with open("x_val.txt", "r") as f:
    x_val = [filename[:-1] for filename in f.readlines()]
with open("y_val.txt", "r") as f:
    y_val = [filename[:-1] for filename in f.readlines()]

In [5]:
trainGen = DataGen(x_train, y_train, batch_size=60)
valGen = DataGen(x_val, y_val, batch_size=60)

In [6]:
from tensorflow.keras.metrics import MeanIoU
class mIoU(MeanIoU):
    def __init__(self, num_classes):
        super().__init__(num_classes=num_classes, name="mIoU")
    
    def __call__(self, y_true, y_pred, sample_weight=None):
        y_true = tf.argmax(y_true, axis=-1)
        y_pred = tf.argmax(y_pred, axis=-1)
        return super().__call__(y_true, y_pred, sample_weight=sample_weight)

In [7]:
model = LikeUnet()
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=[mIoU(num_classes=2)])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [8]:
from datetime import datetime
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
logdir = "logs\\" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=logdir, update_freq='batch', profile_batch=0)
filepath = "models\\night-{epoch:02d}-{val_mIoU:.4f}.h5"
checkpoint_callback = ModelCheckpoint(filepath, monitor='val_mIoU', save_weights_only=True)

In [9]:
model.fit_generator(generator=trainGen, validation_data=valGen, epochs=15, workers=8, shuffle=True,
                    callbacks=[tensorboard_callback, checkpoint_callback])

Epoch 1/15
500/500 [==============================] - 445s 890ms/step - loss: 0.2819 - mIoU: 0.8024 - val_loss: 0.1424 - val_mIoU: 0.8949
Epoch 2/15
500/500 [==============================] - 426s 852ms/step - loss: 0.1065 - mIoU: 0.9237 - val_loss: 0.1542 - val_mIoU: 0.8871
Epoch 3/15
500/500 [==============================] - 423s 846ms/step - loss: 0.0793 - mIoU: 0.9441 - val_loss: 0.0841 - val_mIoU: 0.9398
Epoch 4/15
500/500 [==============================] - 425s 850ms/step - loss: 0.0669 - mIoU: 0.9535 - val_loss: 0.0714 - val_mIoU: 0.9507
Epoch 5/15
500/500 [==============================] - 412s 824ms/step - loss: 0.0591 - mIoU: 0.9593 - val_loss: 0.0725 - val_mIoU: 0.9500
Epoch 6/15
500/500 [==============================] - 413s 826ms/step - loss: 0.0537 - mIoU: 0.9636 - val_loss: 0.0649 - val_mIoU: 0.9540
Epoch 7/15
500/500 [==============================] - 401s 802ms/step - loss: 0.0506 - mIoU: 0.9662 - val_loss: 0.0621 - val_mIoU: 0.9589
Epoch 8/15
500/500 [==============

In [10]:
# Small dataset for debugging
# model.fit_generator(generator=valGen, epochs=2, workers=8, shuffle=True)

In [11]:
with open('night.json', 'w') as f:
    f.write(model.to_json())

In [12]:
with open('night.json', 'r') as f:
    json_string = f.read()
model = model_from_json(json_string)